# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#Load csv file
weather_api_df = pd.read_csv("output_data/cities.csv")
weather_api_df.head()

,Unnamed: 0,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,São João da Barra,-21.64,-41.05,91.400,38,57,9.171454,BR,1594660774
1,1,Palafrugell,41.92,3.16,77.000,73,40,3.355410,ES,1594660774
2,2,Wadena,46.44,-95.14,77.000,65,1,18.342908,US,1594660774
3,3,Jamestown,42.10,-79.24,73.004,68,75,13.869028,US,1594660688
4,4,Visby,57.64,18.30,68.000,63,43,8.052984,SE,1594660774


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
#Set latitude and longitude as variable
locations = weather_api_df[["Latitude", "Longitude"]]

#Set humidity as variable and convert to float
humidity = weather_api_df["Humidity"].astype(float)

#Create heatmap using gmaps
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#Narrow down dataframe to fit new weather parameters
ideal_cities_df = weather_api_df.loc[(weather_api_df["Max Temperature"]>50) & (weather_api_df["Max Temperature"]<60)
                             & (weather_api_df["Wind Speed"]<10) & (weather_api_df["Cloudiness"]<60)
                             & (weather_api_df["Humidity"]<70)]
ideal_cities_df

,Unnamed: 0,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
53,53,Mercedes,-34.65,-59.43,53.006,36,32,5.994999,AR,1594660781
181,181,Khatanga,71.97,102.50,58.766,61,26,8.455633,RU,1594660799
212,212,Vanderhoof,54.02,-124.02,59.468,51,0,9.976752,CA,1594660804
238,238,Mount Isa,-20.73,139.50,51.800,53,15,6.934514,AU,1594660809
281,281,Chokurdakh,70.63,147.92,54.752,65,43,9.730689,RU,1594660815
301,301,Eshowe,-28.89,31.47,55.580,42,0,3.265932,ZA,1594660817
304,304,Okhotsk,59.38,143.30,59.756,68,27,4.764682,RU,1594660794
346,346,Whitehorse,60.72,-135.05,59.000,62,40,8.052984,CA,1594660698
391,391,Rusape,-18.53,32.13,54.176,47,0,2.773806,ZW,1594660829
428,428,Yeppoon,-23.13,150.73,51.800,57,0,8.052984,AU,1594660834


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
#Create list to populate with hotel search
hotel = []

#Set variables for parameters in the loop
target_type = "lodging" 
radius = 5000

#Set base url to variable
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#Create loop to go through dataframe using row lat/long coordinates and general search parameters
for index,row in ideal_cities_df.iterrows():
    coords = f"{row['Latitude']},{row['Longitude']}"
    params = {
    "location": coords,
    "types": target_type,
    "radius": radius,
    "key": g_key
    }
    response = requests.get(base_url, params).json()
    #Create exceptions for cities that do not return hotel results
    try:
        hotel_name = response['results'][0]['name']
        hotel.append(hotel_name)
        print(f"Found {hotel_name} in {row['City']}")
    except:
        #If results not found, populate list with "Not Found"
        hotel.append("Not Found")
        print(f"Hotel not found for {row['City']}")

Found Posada del Inti Centro in Mercedes
Found Gostinitsa "Zapolyar'ye" in Khatanga
Found Vanderhoof Inn in Vanderhoof
Found ibis Styles Mt Isa Verona in Mount Isa
Hotel not found for Chokurdakh
Found Sugar Hill Manor in Eshowe
Hotel not found for Okhotsk
Found Westmark Hotel in Whitehorse
Found Surrey Lodge in Rusape
Found Villa Mar Colina in Yeppoon
Found Vandrehuset in Nuuk
Found River Side Lodge in Masvingo
Found Dobro Pozhalovaty in Usinsk
Found SERENJE SOCH INN in Serenje


In [7]:
#Store in new variable, and set column names to match code in next cells
hotel_df = ideal_cities_df
hotel_df['Hotel Name'] = hotel
hotel_df.rename(columns={"Latitude": "Lat", "Longitude": "Lng"}, inplace=True)
hotel_df

C:\Users\Alex\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Alex\anaconda3\envs\PythonData\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Unnamed: 0,City,Lat,Lng,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
53,53,Mercedes,-34.65,-59.43,53.006,36,32,5.994999,AR,1594660781,Posada del Inti Centro
181,181,Khatanga,71.97,102.50,58.766,61,26,8.455633,RU,1594660799,"Gostinitsa ""Zapolyar'ye"""
212,212,Vanderhoof,54.02,-124.02,59.468,51,0,9.976752,CA,1594660804,Vanderhoof Inn
238,238,Mount Isa,-20.73,139.50,51.800,53,15,6.934514,AU,1594660809,ibis Styles Mt Isa Verona
281,281,Chokurdakh,70.63,147.92,54.752,65,43,9.730689,RU,1594660815,Not Found
301,301,Eshowe,-28.89,31.47,55.580,42,0,3.265932,ZA,1594660817,Sugar Hill Manor
304,304,Okhotsk,59.38,143.30,59.756,68,27,4.764682,RU,1594660794,Not Found
346,346,Whitehorse,60.72,-135.05,59.000,62,40,8.052984,CA,1594660698,Westmark Hotel
391,391,Rusape,-18.53,32.13,54.176,47,0,2.773806,ZW,1594660829,Surrey Lodge
428,428,Yeppoon,-23.13,150.73,51.800,57,0,8.052984,AU,1594660834,Villa Mar Colina


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content=hotel_info, display_info_box=True)

fig.add_layer(markers)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))